https://huggingface.co/course/chapter1/3?fw=pt

In [1]:
import numpy as np
#possibly replace with e.g. util.pytorch_cos_sim from sentence_transformers
from scipy.spatial.distance import cosine

from transformers import pipeline


2022-05-21 05:46:55.376716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-21 05:46:55.376746: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Tokenizer and models

DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT’s performances as measured on the GLUE language understanding benchmark.

https://huggingface.co/docs/transformers/model_doc/distilbert

The abstract from the paper is the following:

> As Transfer Learning from large-scale pre-trained models becomes more prevalent in Natural Language Processing (NLP), operating these large models in on-the-edge and/or under constrained computational training or inference budgets remains challenging. In this work, we propose a method to pre-train a smaller general-purpose language representation model, called DistilBERT, which can then be fine-tuned with good performances on a wide range of tasks like its larger counterparts. While most prior work investigated the use of distillation for building task-specific models, we leverage knowledge distillation during the pretraining phase and show that it is possible to reduce the size of a BERT model by 40%, while retaining 97% of its language understanding capabilities and being 60% faster. To leverage the inductive biases learned by larger models during pretraining, we introduce a triple loss combining language modeling, distillation and cosine-distance losses. Our smaller, faster and lighter model is cheaper to pre-train and we demonstrate its capabilities for on-device computations in a proof-of-concept experiment and a comparative on-device study.

In [2]:
from transformers import AutoTokenizer

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1

2022-05-21 05:47:01.660918: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-21 05:47:01.660975: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (zeno): /proc/driver/nvidia/version does not exist
2022-05-21 05:47:01.662742: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [4]:
#What about slang akronyms ?
IDs = tokenizer(["OMG, this movie", "WTF is going"], padding=True, truncation=True, return_tensors="tf")
print(IDs)
print(tokenizer.tokenize("OMG"))
print(tokenizer.tokenize("LOL"))
print(tokenizer.tokenize("BERT"))
print(tokenizer.tokenize("Interdisciplinary"))
print(tokenizer.tokenize("comprehensibility"))

#notice that the full BERT tokenizer would split the following into sub words:
print(tokenizer.tokenize("transformers"))

#the splitting into multiple tokens may help to identify new words which include similar tokens

{'input_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[  101, 18168,  2290,  1010,  2023,  3185,   102],
       [  101,  1059, 24475,  2003,  2183,   102,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0]], dtype=int32)>}
['om', '##g']
['lo', '##l']
['bert']
['interdisciplinary']
['com', '##pre', '##hen', '##sibility']
['transformers']


Notice the identical tokens for identical words

In [5]:
raw_inputs = [
    "How does the dimension of the embedding come about",
    "How do you come around?",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[ 101, 2129, 2515, 1996, 9812, 1997, 1996, 7861, 8270, 4667, 2272,
        2055,  102],
       [ 101, 2129, 2079, 2017, 2272, 2105, 1029,  102,    0,    0,    0,
           0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]], dtype=int32)>}


Notice the different ways of calling the tokenizer:

In [6]:
s = "BIPM alumni are in high demand!"
tokens = tokenizer.tokenize(s)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

ids2 = tokenizer(s, padding=True, truncation=True, return_tensors="tf")
print(ids2)

[12170, 9737, 9441, 2024, 1999, 2152, 5157, 999]
{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101, 12170,  9737,  9441,  2024,  1999,  2152,  5157,   999,
          102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


#### Decoding tokens

Note that the decode method not only converts the indices back to tokens, but also groups together the tokens that were part of the same words to produce a readable sentence. 

In [7]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

bipm alumni are in high demand!


### Embedding

In [8]:
from transformers import TFAutoModel

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
model = TFAutoModel.from_pretrained(checkpoint)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [9]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)
#2 sentences, 13 as longest token, 768 as shape of each embedding

(2, 13, 768)


So the output is high-dimensional (embedding of dimension 768 for each individual word!)

Let's find out if the embedding is really contextual, i.e. whether identical single words are different depending on the sentence:

In [10]:
#How is the first word of both sentences
raw_inputs = [
    "How does the dimension of the embedding come about",
    "How do you come around?",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
outputs = model(inputs)

How_embed_1 = outputs.last_hidden_state[0,0,:]
How_embed_2 = outputs.last_hidden_state[1,0,:]
print(np.sum(How_embed_1==How_embed_2)) #counts how many of the 768 vectors are the same (0)

print("similarity:")
print( 1 - cosine(How_embed_1, How_embed_2) ) 
#most_similar=data_

0
similarity:
0.9125812649726868


### Tasks:

1. Embedding Similarity 
  * Write a function that takes as input two sentences, finds identical words, computes their contextual embeddings and prints the cosine similarity.
  * Ask interesting questions (case, punctuation,...)
  * Use longer words and find similarity between sub words

2. Load the IMBD reviews and sample 500 positive and negative reviews. 
  * Extract the embedding vectors
  * Fit a Naive Bayes classifier

In [11]:
##Task 1
def similarity_check(sent1, sent2):
    split1 = sent1.split()
    split2 = sent2.split()
    same_words = []
    for i in split1: 
        if split2.count(i) > 0: same_words.append([split1.index(i), split2.index(i), i])
    
    raw_inputs = [sent1, sent2]
    inputs = tokenizer(raw_inputs, return_tensors="tf")
    outputs = model(inputs)

    similarity = []
    for index, i in enumerate(same_words):
        embedd_1 = outputs.last_hidden_state[0,same_words[index][0], :]
        embedd_2 = outputs.last_hidden_state[1,same_words[index][1], :]
        similarity.append([same_words[index][2], 1 - cosine(embedd_1, embedd_2)])
    return similarity

In [12]:
sent1 = "work to live"
sent2 = "live to work"
similarity_check(sent1, sent2)

[['work', 0.13984420895576477],
 ['to', 0.5869045853614807],
 ['live', 0.3014388680458069]]

In [13]:
##Task 2
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)
full_docs = np.concatenate((train_data, test_data))

#turn all keys to words
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])
dec_reviews = ["" for x in range(full_docs.shape[0])]
for j in range(full_docs.shape[0]):
    dec_reviews[j] = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in full_docs[j]])

#get 500 positive and 500 negative reviews (since the train_labels are perfectly stratified we can just train_test)
train_decoded = dec_reviews[:25000]
x_used, x_extra, y_used, y_extra = train_test_split(train_decoded, train_labels, test_size = 0.96, stratify = train_labels)

In [14]:
inputs = tokenizer(x_used, padding=True, truncation=True, return_tensors="tf")
outputs = model(inputs)

#create embeddings
x_embed = outputs.last_hidden_state

In [15]:
x_embed.shape #this neds to be reshaped

TensorShape([1000, 512, 768])

In [24]:
reshaped = np.array(x_embed).reshape(1000, -1)
reshaped.shape #better

(1000, 393216)

In [26]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
gnb = GaussianNB()
gnb.fit(reshaped, y_used) #fit to GaussianNB (because CategoricalNB cant handle negative values)
accuracy_score(y_used, gnb.predict(reshaped)) #get train accuracy
#90.4% Training accuracy 

0.904